In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('''data/uber_data.csv''')

In [6]:
df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80
5,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,5.92,-74.017197,40.705383,1,N,-73.978073,40.755787,1,23.5,1.0,0.5,5.06,0.00,0.3,30.36
6,2,2016-03-01 00:00:00,2016-03-01 00:00:00,6,5.72,-73.994583,40.727848,1,N,0.000000,0.000000,2,23.0,0.5,0.5,0.00,0.00,0.3,24.30
7,1,2016-03-01 00:00:01,2016-03-01 00:16:04,1,6.20,-73.788773,40.647758,1,N,-73.829208,40.712345,3,20.5,0.5,0.5,0.00,0.00,0.3,21.80
8,1,2016-03-01 00:00:01,2016-03-01 00:05:00,1,0.70,-73.958221,40.764641,1,N,-73.967896,40.762901,1,5.5,0.5,0.5,2.00,0.00,0.3,8.80
9,2,2016-03-01 00:00:01,2016-03-01 00:24:06,3,7.18,-73.985779,40.741192,1,N,-73.946350,40.797878,1,23.5,0.5,0.5,3.20,0.00,0.3,28.00


#### Task 2: Design a Data Model and create Fact and Dimension Tables

In [8]:
import datetime 
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [9]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [10]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

#### Now we will create dimension Tables 

**1. DateTime Dimension**

In [12]:
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

In [15]:
datetime_dim.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pick_hour',
       'pick_day', 'pick_month', 'pick_year', 'pick_weekday', 'drop_hour',
       'drop_day', 'drop_month', 'drop_year', 'drop_weekday', 'datetime_id'],
      dtype='object')

In [14]:
# Primary Key, -- to be used to join back data with fact table
datetime_dim['datetime_id'] = datetime_dim.index

In [16]:
datetime_dim = datetime_dim[[
        'datetime_id', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pick_hour',
       'pick_day', 'pick_month', 'pick_year', 'pick_weekday', 'drop_hour',
       'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [17]:
datetime_dim.head(5)

,datetime_id,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2016-03-01,2016-03-01 00:07:55,0,1,3,2016,1,0,1,3,2016,1
1,1,2016-03-01,2016-03-01 00:11:06,0,1,3,2016,1,0,1,3,2016,1
2,2,2016-03-01,2016-03-01 00:31:06,0,1,3,2016,1,0,1,3,2016,1
3,3,2016-03-01,2016-03-01 00:00:00,0,1,3,2016,1,0,1,3,2016,1
4,4,2016-03-01,2016-03-01 00:00:00,0,1,3,2016,1,0,1,3,2016,1


**2. Passanger Count Dimension**

In [19]:
def occupancy_category(value):
    if value == 1:
        return "Single Person"
    if value >= 2 and value < 4:
        return "Light Occupancy"
    if value >= 4:
        return "High Occupancy"
    
    return "Invalid/Missing"

occupancy_dim = df[['passenger_count']].reset_index(drop=True)
occupancy_dim['occupancy_dim_id'] = occupancy_dim.index

occupancy_dim['occupancy_code'] = df['passenger_count'].apply(occupancy_category)
occupancy_dim = occupancy_dim[['occupancy_dim_id', 'passenger_count', 'occupancy_code']]

In [20]:
occupancy_dim.head(5)

,occupancy_dim_id,passenger_count,occupancy_code
0,0,1,Single Person
1,1,1,Single Person
2,2,2,Light Occupancy
3,3,3,Light Occupancy
4,4,5,High Occupancy


**3. Trip Distance Dimension**

In [22]:
def trip_distance_category(value):
    if value >= 0 and value < 15:
        return "Small Distance/City Blocks"
    
    if value >= 15 and value < 40:
        return "Medium Distance/Across City"
    
    if value > 40:
        return "Large Distance/Intercity"
    
    return "Invalid/Missing"


trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index

trip_distance_dim['trip_distance_code'] = df['trip_distance'].apply(trip_distance_category)
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance', 'trip_distance_code']]

In [23]:
trip_distance_dim.head(5)

,trip_distance_id,trip_distance,trip_distance_code
0,0,2.50,Small Distance/City Blocks
1,1,2.90,Small Distance/City Blocks
2,2,19.98,Medium Distance/Across City
3,3,10.78,Small Distance/City Blocks
4,4,30.43,Medium Distance/Across City


**4. Rate Code Dimension**

In [25]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

In [26]:
rate_code_dim.head(5)

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,3,Newark


**5. PickUp & DropOff Dimension**

In [27]:
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 


dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]

**6. Payment Type Dimension**

In [28]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

#### Creating Schema [Combining Fact & Dimension Tables]

In [35]:
fact_table = df.merge(occupancy_dim, left_on='trip_id', right_on='occupancy_dim_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'occupancy_dim_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [39]:
fact_table = df[['passenger_count', 'trip_distance']]

In [40]:
fact_table.head(10)

,passenger_count,trip_distance
0,1,2.50
1,1,2.90
2,2,19.98
3,3,10.78
4,5,30.43
5,5,5.92
6,6,5.72
7,1,6.20
8,1,0.70
9,3,7.18
